In [1]:
%matplotlib qt
import SingleOrigin as so
import numpy as np
import pandas as pd
import tqdm
from matplotlib import pyplot as plt
from matplotlib.patches import Ellipse
from matplotlib import colors as colors
import matplotlib as mpl
from matplotlib_scalebar.scalebar import ScaleBar
import PyQt5
from scipy import ndimage

First we designate the zone axis for the image and two in-plane basis vectors

In [3]:
za = [1,1,0]          #Zone Axis direction
a1 = [-1,1,0]         #Apparent horizontal axis in projection
a2 = [0,0,1]          #Most vertical axis in projection

Next we load a .cif file to build a projected unit cell template for the image we will analyze.

In [4]:
uc = so.UnitCell()

uc.transform_basis(za, a1, a2)

"""Project Unit Cell to 2D and combine coincident/proximate columns"""
uc.project_uc_2d(proj_axis = 0, ignore_elements = []) 

uc.combine_prox_cols(toler = 1e-2)
uc.at_cols.loc[:, 'LatticeSite'] = ['B', 'O', 'A']

uc.plot_unit_cell(label_by='LatticeSite')
sites = sorted(uc.at_cols.loc[:, 'LatticeSite'].to_list())

transformed lattice parameters: 
 a1:     5.557 
 a2:     5.557 
 a3:     3.9267 
 alpha:  90.0 
 beta:   90.0 
 gamma:  90.0 

Distances (in Angstroms) being combined...
0.0


We import the image and any associated metadata

In [6]:
image, metadata = so.import_image(display_image=True)

Create an AtomicColumnLattice object using the unit cell projection we built and and the image we want to analize. We will then perform various operations on this object to locate the atomic columns and visualize the results. All the resulting data will be held within the object.

In [7]:
acl = so.AtomicColumnLattice(image, uc, probe_fwhm=0.8,
                             xlim=None, ylim=None)

The first operation we perform is to find basis vectors by using the FFT. To do this you must select two reciprocal lattice points (spots in the FFT) corresponding to the basis vectors you specified in cell 3 as "a1" and "a2". These must be picked in the correct order (i.e. "a1" then "a2"). If the spots are not present or weak, select higher order spots (e.g. if "a2" is the [001] direction but the 001 spot is a forbidden reflection, so you should specify "a2_order=2" in the following function and select an 002 spot in the FFT.) In this example [110] and [001] are both allowed reflections and can be easily located in the FFT.

In [8]:
acl.fft_get_basis_vect(a1_order=1, a2_order=1, sigma=2)

done selecting 

